In [ ]:
import os
OUTPUT_DIR = os.path.join("lips","Outputs")
INPUT_DIR = os.path.join("lips", "Outputs", "Data", "test")
BENCHMARK_DIR = os.path.join("lips", "Outputs", "Benchmark 1")

In [ ]:
import numpy as np
from pprint import pprint
from lips.dataset import PowerGridDataSet
from lips.simulators import DCApproximation
from lips.benchmark import Benchmark
from lips.evaluation import Evaluation
from lips.simulators import FullyConnectedNN
from pprint import pprint

In [ ]:
# Run the cell below to produce a table of contents 
res = None
try:
    from jyquickhelper import add_notebook_menu
    res = add_notebook_menu()
except ModuleNotFoundError:
    print("Impossible to automatically add a menu / table of content to this notebook.\nYou can download \"jyquickhelper\" package with: \n\"pip install jyquickhelper\"")
res

## Data Generation
To generate the datasets, we use `PowerGridDataSet` class which is a sub-class of a more general `DataSet` class and is designed specifically for power grid data. It allows to generate one dataset at a time. We keep the DataSet class (therefore the platform) as generic as possible for its use in other research and industrial domains. 

In [ ]:
dataset = PowerGridDataSet(env_name="l2rpn_case14_sandbox")

In [ ]:
test_data = dataset.generate(nb_samples=int(1024) * 128,
                             tag="test",
                             val_regex=".*99[0-9].*",
                             use_lightsim_if_available=True,
                             reference_number=0,
                             agent_generator_name="random_nn1",
                             agent_parameters={"p": 0.5},
                             skip_gameover=True,
                             env_seed=1234,
                             agent_seed=14,
                             verbose=True
                            )

#### save the generated dataset
<span style="color:red">In this version, I have stored a list of Grid2op `Observations`, but it takes a lot of space on disk !!! </span>

In [ ]:
dataset.save(path=OUTPUT_DIR)

#### Load the generated dataset 

In [ ]:
test_data = dataset.load(INPUT_DIR)

## Loading and evaluating a Simulator

### Physical Simulator (DC approximation)
To assess the performance of a physical solver, one can use the DCApproximation available in `simulators` module. As opposed to augmented simulators, it does not require to be learned. Hence, only test dataset should be used to evaluate its performance. Various variables could be used as inputs and outputs of this solver.

In [ ]:
dc_app = DCApproximation(dataset.env._init_grid_path,
                         name="dc_approx",
                         attr_x=("prod_p", "prod_v", "load_p", "load_q", "topo_vect"),  # input that will be given to the proxy
                         attr_y=("a_or", "a_ex", "p_or", "p_ex", "q_or", "q_ex", "prod_q", "load_v", "v_or", "v_ex"),
                        )

DC approximator should be initialized with generated data. 

In [ ]:
# I have reduced the size of experimentation, as it takes long time to be excuted for DC approximation
dc_app.init(test_data[:5000])

Once, intitialized, it allows to infer the output variables from the input ones using its `predict` function.

In [ ]:
predictions, observations, predict_time = dc_app.predict(save_path=BENCHMARK_DIR)

<<<<<<< local <removed>


100%|█████████████████████████████████████████████████████████████| 5000/5000 [01:55<00:00, 43.45it/s]


>>>>>>> remote <modified: >


## Evaluation metrics
The `Evaluation` class allows to evaluate the performance of different simulators with respect to various criteria arosed from various domains. 
- Physic compliance
- Machine Learning metrics
- Generalization
- Readiness (not yet implemented)

In this new version of platform, one should activate or deactivate the verifications required for a specific benchmark, before passing an object of this class to the `Benchmark` class.

we start by creating an object of the Evaluation class as follows:

In [ ]:
evaluation = Evaluation()

Then, a dictionary of Metric Activation is provided using `get_active_dict()` function : 

In [ ]:
active_dict = evaluation.get_active_dict()

In [ ]:
pprint(active_dict)

#### Activate the required verifications 

For example, if we wish for benchmark 1 only test the ML metrics and to verify the positivity of current values, one possible way to active them is the following :

In [ ]:
active_dict["evaluate_ML"] = True
active_dict["evaluate_physic"]["verify_current_pos"] = True
evaluation.set_active_dict(active_dict)

In [ ]:
pprint(evaluation.get_active_dict())

It can be verified as above, that these two tests are activated for the Benchmark.

## Benchmark
Once a dataset has been generated, a simulator has been trained and predicted the required results and an evaluator with required metrics is parameterized, the `Benchmark` class allows to assess the performance of the simulator with respect to the indicated metrics and criteria. 

As it can be seen below, the Benchmark class, takes as input an object of the following classes:

- PowerGridDataSet
- DCApproximation
- Evaluation

In [ ]:
benchmark_dc = Benchmark("Risk assessment", dataset, dc_app, evaluation, save_path=BENCHMARK_DIR)

The prediction time of the corresponding simulator could be accessed rapidly.

In [ ]:
benchmark_dc.prediction_time

53.3128616809845

Now, we can evaluate the simulator using all the indicated metrics in the previous section by activating the metrics. The choice allows to consider the real observations for physic compliance verifications. However, for Machine Learning metrics, the verification does not change by this parameter.

In [16]:
BENCHMARK_DIR

'lips\\Outputs\\Benchmark 1'

In [ ]:
benchmark_dc.evaluate_simulator(choice="real",
                                EL_tolerance=0.04,
                                LCE_tolerance=1e-3,
                                KCL_tolerance=1e-2,
                                active_flow=True,
                                save_path=os.path.join(BENCHMARK_DIR, "real")
                               )

All the metrics could be accessed via the member variables of Benchmark class as follows : 

#### ML metrics

In [ ]:
print("The list of all the available metrics")
pprint(benchmark_dc.metrics_ML.keys())
print("********************")
print("MAE_avg")
pprint(benchmark_dc.metrics_ML["MAE_avg"])
print("MAPE")
pprint(benchmark_dc.metrics_ML["mape_avg"])
print("MAPE90")
pprint(benchmark_dc.metrics_ML["mape90"])

#### Physic compliances 

In [ ]:
benchmark_dc.metrics_physics

as there is no violation of current values, this metric does not contain any value.

The Physic compliance verification could also be performed for predictions.

In [ ]:
benchmark_dc.evaluate_simulator(choice="predictions",
                                EL_tolerance=0.04,
                                LCE_tolerance=1e-3,
                                KCL_tolerance=1e-2,
                                active_flow=True,
                                save_path=os.path.join(BENCHMARK_DIR, "predictions")
                               )

As it can be seen there is no violation for the positivity of current values.

# <span style="color:red">Critical evaluation</span>

Here, I activate the EL, LCE and KCL metrics from physic compliance category.

In [ ]:
active_dict = evaluation.get_active_dict()

In [ ]:
active_dict

In [ ]:
active_dict["evaluate_physic"]["verify_EL"] = True
active_dict["evaluate_physic"]["verify_LCE"] = True
active_dict["evaluate_physic"]["verify_KCL"] = True

In [ ]:
evaluation.set_active_dict(active_dict)

In [ ]:
pprint(evaluation.get_active_dict())

### <span style="color:red">Now Evaluating these criteria using Benchmark class</span>

In [ ]:
benchmark_dc = Benchmark("Risk assessment", dataset, dc_app, evaluation, save_path=BENCHMARK_DIR)

In [ ]:
benchmark_dc.evaluate_simulator(choice="real",
                                EL_tolerance=0.04,
                                LCE_tolerance=1e-3,
                                KCL_tolerance=1e-2,
                                active_flow=True,
                                save_path=os.path.join(BENCHMARK_DIR, "real")
                               )

There is no problem using Real Observations

In [ ]:
benchmark_dc.evaluate_simulator(choice="predictions",
                                EL_tolerance=0.04,
                                LCE_tolerance=1e-3,
                                KCL_tolerance=1e-2,
                                active_flow=True,
                                save_path=os.path.join(BENCHMARK_DIR, "predictions")
                               )

<<<<<<< local <removed>


************* Basic verifier *************
Current positivity check passed for origin side !
----------------------------------------------
Current positivity check passed for extremity side !
----------------------------------------------
************* Check loss *************
Verification is done without any violation !
************* Check Energy Conservation *************
Number of failed cases is 5000 and the proportion is 100.000% : 
************* Check kirchhoff's current law *************
7.14% not verify the Kirchhoff's current law at 0.01 tolerance


>>>>>>> remote <modified: >


It can be seen here that, therea 100% of the cases that DC violates the Law of Conservation of Energy and 7% of the cases that it violates the Kirchoff's current law.

### <span style="color:red">I dont think that it could be a problem from the implementation point of view !</span>

In [27]:
evaluation.metrics_physics

{'BasicVerifications': {'currents': {}},
 'EL': {'violation_percentage': 0.0,
  'EL_values': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)},
 'LCE': {'violation_percentage': 100.0,
  'LCE_values': array([4.22937  , 5.1879272, 4.0081177, ..., 2.707962 , 2.7460938,
         2.692276 ], dtype=float32)},
 'KCL': {'violation_percentage': 7.142857142857142,
  'nodes_values': array([[ 4.2293701e+00,  1.9073486e-06,  0.0000000e+00, ...,
          -4.7683716e-07,  0.0000000e+00,  0.0000000e+00],
         [ 5.1879349e+00,  1.9073486e-06,  0.0000000e+00, ...,
          -4.7683716e-07,  9.5367432e-07,  0.0000000e+00],
         [ 4.0081100e+00, -5.7220459e-06,  0.0000000e+00, ...,
           2.3841858e-07, -9.5367432e-07,  0.0000000e+00],
         ...,
         [ 2.7079487e+00,  1.9073486e-06,  1.9073486e-06, ...,
           0.0000000e+00, -1.9073486e-06,  0.0000000e+00],
         [ 2.7460823e+00,  0.0000000e+00,  0.0000000e+00, ...,
           2.3841858e-07, -1.9073486e-06,  0.0000000e+00],
